In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import namedtuple
Dataset = namedtuple("Dataset", "data target")

from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate, cross_val_predict

from sklearn.linear_model import (ARDRegression, BayesianRidge, ElasticNet, ElasticNetCV, HuberRegressor, Lars, LarsCV,
                                  Lasso, LassoCV, LassoLars, LassoLarsCV, LassoLarsIC, LinearRegression,
                                  MultiTaskElasticNet, MultiTaskElasticNetCV, MultiTaskLasso, MultiTaskLassoCV,
                                  OrthogonalMatchingPursuit, OrthogonalMatchingPursuitCV, PassiveAggressiveRegressor,
                                  RANSACRegressor, Ridge, RidgeCV, SGDRegressor, TheilSenRegressor)

from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor)
from sklearn.cross_decomposition import (CCA, PLSCanonical, PLSRegression)
from sklearn.tree import (DecisionTreeRegressor, ExtraTreeRegressor)
from sklearn.gaussian_process import (GaussianProcessRegressor)
from sklearn.neighbors import (KNeighborsRegressor, RadiusNeighborsRegressor)
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import (LinearSVR, NuSVR, SVR)

def RMSE(actual,predicted):
    return np.sqrt(metrics.mean_squared_error(np.nan_to_num(np.log1p(actual)), np.nan_to_num(np.log1p(predicted))))

In [2]:
IN_TRAIN = 'in/train-wrangled.csv'

In [3]:
%%time
dataframe = pd.read_csv(IN_TRAIN, dtype={'fullVisitorId': 'str'}, low_memory=False)

In [103]:
print("Number of unique users: {}".format(len(dataframe)))

Number of unique users: 714167


In [217]:
# train_df, _ = train_test_split(dataframe, train_size=10000, shuffle=True, random_state=42)
train_df = dataframe

In [218]:
train_df = train_df.drop(columns=["fullVisitorId", "sessionId", "visitId", "visitStartTime"])
train_df = train_df.fillna(0)
train_df.head()

,date,visitNumber,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,...,geoNetwork.subContinent Western Asia,geoNetwork.subContinent Western Europe,trafficSource.source (direct),trafficSource.source Other,trafficSource.source Partners,trafficSource.source analytics.google.com,trafficSource.source google,trafficSource.source mall.googleplex.com,trafficSource.source youtube.com,totals.transactionRevenue
0,0.0219,0.000000,0,0.0,0.020040,0.333333,0.001923,0.00000,0,0,...,0,0,0,0,0,0,1,0,0,0.0
1,0.0329,0.000000,0,0.0,0.032064,0.333333,0.003125,0.00000,0,0,...,0,0,0,0,0,0,1,0,0,0.0
2,0.9410,0.000000,0,0.0,0.010020,0.333333,0.001202,0.00000,0,0,...,0,0,0,0,0,0,1,0,0,0.0
3,0.9526,0.000000,0,0.0,0.002004,0.333333,0.000481,0.00000,0,0,...,0,0,0,0,0,0,0,0,1,0.0
4,0.9410,0.000032,0,0.0,0.002004,0.333333,0.001202,0.00361,0,0,...,0,0,0,0,0,0,1,0,0,0.0


In [219]:
def load_csv_dataset(df):
    target = df["totals.transactionRevenue"].values
    data = df.drop(columns="totals.transactionRevenue").values
    return Dataset(data=data, target=target)
dataset = load_csv_dataset(train_df)

In [201]:
g_params = { "l1_ratio": [0, 0.2, 0.5, 0.8, 1], "fit_intercept":[True], "normalize":[False], "selection": ["random"]}

estimator = ElasticNet()

In [165]:
g_params = {"epsilon": [0.3], "C":[1.2], "loss": ["epsilon_insensitive","squared_epsilon_insensitive"], "fit_intercept":[True]}

estimator = LinearSVR()

In [220]:
# LinearSVR
estimator = LinearSVR(epsilon=0.3, C=1.2, loss="epsilon_insensitive", fit_intercept=True)

In [202]:
%%time
scorer = metrics.make_scorer(RMSE, greater_is_better=False)

gs = GridSearchCV(estimator, g_params, scoring=scorer)
gs.fit(dataset.data, dataset.target)

/home/eugenius/.local/share/virtualenvs/scikit-XsFrGcKX/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/eugenius/.local/share/virtualenvs/scikit-XsFrGcKX/lib/python3.5/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in log1p
/home/eugenius/.local/share/virtualenvs/scikit-XsFrGcKX/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/eugenius/.local/share/virtualenvs/scikit-XsFrGcKX/lib/python3.5/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in log1p
/home/eugenius/.local/share/virtualenv

CPU times: user 31.8 s, sys: 3.33 s, total: 35.1 s
Wall time: 30.6 s


/home/eugenius/.local/share/virtualenvs/scikit-XsFrGcKX/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [167]:
gs.best_score_, gs.best_params_

(-2.228336434572849,
 {'C': 1.2,
  'epsilon': 0.3,
  'fit_intercept': True,
  'loss': 'epsilon_insensitive'})

In [221]:
%%time
# estimator.fit(X, y)
# results = estimator.predict(dataset.data)

results = cross_val_predict(estimator, dataset.data, dataset.target)

# results = cross_val_predict(estimator, X, y)

# results = cross_val_predict(gs, dataset.data, dataset.target)

CPU times: user 5.16 s, sys: 3.52 s, total: 8.68 s
Wall time: 10.8 s


In [223]:
results[results < 0] = 0
np.mean(results)

0.12948298417176582

In [224]:
RMSE(dataset.target, results)

2.0657187696129196

In [180]:
len(results[results==0])/len(results), len(dataset.target[dataset.target==0])/len(dataset.target)

(0.0, 0.9842)

In [14]:
estimator.fit(dataset.data, dataset.target)
importances = estimator.feature_importances_
feature_names = np.array(list(train_df)[:-1])
feature_names[importances > 0.05]

array(['date', 'visitNumber', 'totals.pageviews',
       'geoNetwork.city not available in demo dataset',
       'geoNetwork.country Canada'], dtype='<U46')

In [15]:
estimator.fit(X, y)
importances = estimator.feature_importances_
importances

NameError: name 'X' is not defined

In [182]:
f = results[ np.abs(results - dataset.target)/ (dataset.target+1) < 0.1 ]
print(len(f), np.unique(f))
np.unique(f)

2972 [-0.09997882 -0.09994522 -0.09994063 ...  0.09993195  0.09995387
  0.09999627]


array([-0.09997882, -0.09994522, -0.09994063, ...,  0.09993195,
        0.09995387,  0.09999627])